In [1]:
import os
import hsrb_interface
import rospy
import sys
from hsrb_interface import geometry

In [2]:
os.environ['ROS_MASTER_URI'] = 'http://hsrb.local:11311'
os.environ['ROS_IP'] = '169.254.52.4'
print(os.environ['ROS_MASTER_URI'])
print(os.environ['ROS_IP'])

http://hsrb.local:11311
169.254.52.4


In [19]:
# Move timeout[s]
_MOVE_TIMEOUT=60.0
# Grasp force[N]
_GRASP_FORCE=0.2
# TF name of the bottle
_BOTTLE_TF='ar_marker/1'
# TF name of the gripper
_HAND_TF='hand_palm_link'

In [4]:
# Preparation for using the robot functions
robot = hsrb_interface.Robot()
omni_base = robot.get('omni_base')
whole_body = robot.get('whole_body')
gripper = robot.get('gripper')
tts = robot.get('default_tts')

In [11]:
# Posture that 0.02[m] front and rotate -1.57 around z-axis of the bottle maker
bottle_to_hand = geometry.pose(z=-0.02, ek=-1.57)

# Posture to move the hand 0.1[m] up
hand_up = geometry.pose(x=0.1)

# Posture to move the hand 0.5[m] back
hand_back = geometry.pose(z=-0.5)

# Location of the sofa
sofa_pos = (1.2, 0.4, 1.57)

In [6]:
# Greet
rospy.sleep(5.0)
tts.say('Hi. My name is Mobi. I will pick the bottle on the table.')
rospy.sleep(5.0)

In [26]:
try:
    gripper.command(1.0)
    whole_body.move_to_go()
except:
    tts.say('Fail to initialize.')
    rospy.logerr('fail to init')
    sys.exit()

In [24]:
# Transit to initial grasping posture
whole_body.move_to_neutral()
# Look at the hand after the transition
whole_body.looking_hand_constraint = True

In [25]:
try:
    # Transit to initial grasping posture
    whole_body.move_to_neutral()
    # Look at the hand after the transition
    whole_body.looking_hand_constraint = True
    # Move the hand to front of the bottle
    whole_body.move_end_effector_pose(bottle_to_hand, _BOTTLE_TF)
    # Specify the force to grasp
    gripper.apply_force(_GRASP_FORCE)
    # Wait time for simulator's grasp hack. Not needed on actual robot
    rospy.sleep(2.0)
    # Move the hand up on end effector coordinate
    whole_body.move_end_effector_pose(hand_up, _HAND_TF)
    # Move the hand back on end effector coordinate
    whole_body.move_end_effector_pose(hand_back, _HAND_TF)
    # Transit to initial posture
    whole_body.move_to_neutral()
except:
    tts.say('Fail to grasp.')
    rospy.logerr('fail to grasp')
    sys.exit()

In [22]:
%%bash
rosrun rviz rviz  -d rospack find hsrb_common_launch/config/hsrb_display_full_hsrb.rviz

[ INFO] [1724007460.650072726]: rviz version 1.14.25
[ INFO] [1724007460.650115222]: compiled against Qt version 5.12.8
[ INFO] [1724007460.650127941]: compiled against OGRE version 1.9.0 (Ghadamon)
[ INFO] [1724007460.777141755]: Forcing OpenGl version 0.
[ INFO] [1724007461.002312297]: Stereo is NOT SUPPORTED
[ INFO] [1724007461.002342118]: OpenGL device: Mesa Intel(R) UHD Graphics 630 (CFL GT2)
[ INFO] [1724007461.002353455]: OpenGl version: 4.6 (GLSL 4.6) limited to GLSL 1.4 on Mesa system.
